In [1]:
import pandas as pd
import geopandas as gpd
import geohash
import numpy as np
import folium
from shapely.wkt import loads
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon
from IPython.display import display
pd.options.display.max_columns = None
import contextily as cx
import math

### Data paths

In [2]:
input_path = "./../../data/input-data/dish/grid/denver/"
gis_path = "./../../data/input-data/dish/gis/"
output_path = "./../../data/output-data/dish/denver/recommendations/undershooters/"

#### Load Cell Data - Only for plotting purposes

In [3]:
def polysTriangle(lat, lon, bearing, hbeam):
	"""
	Function creates polygons for all cells, if "H_BEAM" = 360 then create circle
	"""
	R = 6378.1
	lat1 = math.radians(lat)
	lon1 = math.radians(lon)

	# Size cell based on 'tech'
	d = 0.5

	# Create LAT/LON points
	lat2 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (hbeam / 2))))

	lon2 = lon1 + math.atan2(math.sin(math.radians(bearing - (hbeam / 2)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

	lat3 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (3 * hbeam / 8))))

	lon3 = lon1 + math.atan2(math.sin(math.radians(bearing - (3 * hbeam / 8)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat3))

	lat4 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (hbeam / 4))))

	lon4 = lon1 + math.atan2(math.sin(math.radians(bearing - (hbeam / 4)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat4))

	lat5 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing)))

	lon5 = lon1 + math.atan2(math.sin(math.radians(bearing))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat5))

	lat6 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (hbeam / 4))))

	lon6 = lon1 + math.atan2(math.sin(math.radians(bearing + (hbeam / 4)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat6))

	lat7 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (3 * hbeam / 8))))

	lon7 = lon1 + math.atan2(math.sin(math.radians(bearing + (3 * hbeam / 8)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat7))

	lat8 = math.asin( math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (hbeam / 2))))

	lon8 = lon1 + math.atan2(math.sin(math.radians(bearing + (hbeam / 2)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat8))

	lat9 = math.asin( math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + 180)))

	lon9 = lon1 + math.atan2(math.sin(math.radians(bearing + 180))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat9))

	# RADIANS to DEGREES
	lat1 = math.degrees(lat1)
	lon1 = math.degrees(lon1)
	lat2 = math.degrees(lat2)
	lon2 = math.degrees(lon2)
	lat3 = math.degrees(lat3)
	lon3 = math.degrees(lon3)
	lat4 = math.degrees(lat4)
	lon4 = math.degrees(lon4)
	lat5 = math.degrees(lat5)
	lon5 = math.degrees(lon5)
	lat6 = math.degrees(lat6)
	lon6 = math.degrees(lon6)
	lat7 = math.degrees(lat7)
	lon7 = math.degrees(lon7)
	lat8 = math.degrees(lat8)
	lon8 = math.degrees(lon8)
	lat9 = math.degrees(lat9)
	lon9 = math.degrees(lon9)

	# If not an omni antenna
	if hbeam < 360:
		return "POLYGON (({} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}))".format(lon1, lat1, lon2, lat2, lon3, lat3, lon4, lat4, lon5, lat5, lon6, lat6, lon7, lat7, lon8, lat8, lon1, lat1)
	else:
		return "POLYGON (({} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}))".format(lon2, lat2, lon3, lat3, lon4, lat4, lon5, lat5, lon6, lat6, lon7, lat7, lon8, lat8, lon9, lat9, lon2, lat2)


In [4]:
gis_df = pd.read_csv("{}gis.csv".format(gis_path), names = ['Name','CILAC', 'SectorID', 'RNC_BSC', 'LAC', 'SectorType', 'Scr_Freq', \
                                                            'UARFCN', 'BSIC', 'Tech', 'Latitude', 'Longitude','Bearing','AvgNeighborDist', \
                                                            'MaxNeighborDist', 'NeighborsCount', 'Eng', 'TiltE','TiltM', 'SiteID', \
                                                            'AdminCellState', 'Asset', 'Asset_Configuration', 'Cell_Type', 'Cell_Name', \
                                                            'City', 'Height', 'RF_Team', 'Asset_Calc', 'Sector_uniq', 'FreqType', 'TAC', \
                                                            'RAC', 'Band', 'Vendor', 'CPICHPwr', 'MaxTransPwr', 'FreqMHz', 'HBW', \
                                                            'VBW', 'Antenna'])
gis_df['geometry'] = gis_df.apply(lambda x: polysTriangle(x['Latitude'], x['Longitude'], x['Bearing'], x['HBW']), axis = 1)
gis_df['geometry'] = gis_df['geometry'].apply(lambda x: shapely.wkt.loads(x))
gis_df = gpd.GeoDataFrame(gis_df, geometry='geometry', crs='EPSG:4326')

gis_df.head(2)

,Name,CILAC,SectorID,RNC_BSC,LAC,SectorType,Scr_Freq,UARFCN,BSIC,Tech,Latitude,Longitude,Bearing,AvgNeighborDist,MaxNeighborDist,NeighborsCount,Eng,TiltE,TiltM,SiteID,AdminCellState,Asset,Asset_Configuration,Cell_Type,Cell_Name,City,Height,RF_Team,Asset_Calc,Sector_uniq,FreqType,TAC,RAC,Band,Vendor,CPICHPwr,MaxTransPwr,FreqMHz,HBW,VBW,Antenna,geometry
0,KNTYS00368A_n70_AWS-4_UL5_1,41036398814,1036398814,-1,-1,401500,405,401500,NaN,NR_Dish,35.720944,-84.022500,0,NaN,NaN,NaN,TYS-13-KNFrndsville,2.0,0,35.7209_-84.0225,1,66392899,NaN,OUTDOOR,440 Dotson Memorial RD,Maryville,53.3400,KN,66392899,66392899#0,F1,25498.0,NaN,n70_AWS-4_UL5,NaN,NaN,45.07,2007.5,61.5,4.935897,KNTYS00368A,"POLYGON ((-84.0225 35.72094, -84.02533 35.7248..."
1,KNTYS00052A_n70_AWS-4_UL5_2,41036402722,1036402722,-1,-1,401500,706,401500,NaN,NR_Dish,35.754778,-83.932083,120,NaN,NaN,NaN,TYS-10-KNMaryville,2.0,0,35.7548_-83.9321,1,2120758710,NaN,OUTDOOR,1002A Grandview Drive,Maryville,24.6888,KN,2120758710,2120758710#120,F1,25498.0,NaN,n70_AWS-4_UL5,NaN,NaN,45.07,2007.5,61.5,4.935897,KNTYS00052A,"POLYGON ((-83.93208 35.75478, -83.92655 35.754..."


#### Load Grid Data

In [5]:
grid_data = pd.read_csv("{}bins_enrichment_dn.csv".format(input_path))
grid_data.head(2)

,grid_cell,grid,cell_name,cilac,avg_rsrp,avg_rsrq,avg_sinr,event_count,grid_event_count,cell_event_count,avg_rsrp_grid,avg_rsrq_grid,avg_sinr_grid,avg_rsrp_cell,avg_rsrq_cell,avg_sinr_cell,perc_cell_events,perc_grid_events,distance_to_cell,grid_max_distance_to_cell,grid_min_distance_to_cell,cell_max_distance_to_cell,perc_cell_max_dist,cell_angle_to_grid,grid_bearing_diff,Scr_Freq,UARFCN,Bearing,TiltE,TiltM,SiteID,City,Height,TAC,Band,Vendor,MaxTransPwr,FreqMHz,HBW,RF_Team,cell_count,same_pci_cell_count,dominance,dist_band
0,9wfvbxf_4602116125,9wfvbxf,DNGJT00002B_n71_F_3,4602116125,-115.0,-12.0,1.0,6.0,6.0,7350.0,-115.0,-12.0,1.0,-105.278164,-11.350908,5.174766,0.000816,1.000000,4607.80642,4607.80642,4607.80642,10684.002855,0.431281,252.74328,47.25672,698,128900,300,2.0,0,39.0381_-108.558,Grand Junction,24.3840,14899,n71_F,NaN,44.77,644.5,63.5,DN,1,1,dominant,tier_2
1,9wfwvmz_4602116192,9wfwvmz,DNGJT00005A_n70_AWS-4_UL15_1,4602116192,-111.0,-10.0,14.0,3.0,9.0,669.0,-108.5,-10.0,15.5,-101.592526,-10.222751,19.678638,0.004484,0.333333,2896.60301,2896.60301,2896.60301,3154.179767,0.918338,40.67767,10.67767,3,401500,30,2.0,0,39.1659_-108.762,Fruita,46.0248,14899,n70_AWS-4_UL15,NaN,46.04,2007.5,61.5,DN,2,2,dominant,tier_3


#### Create GeoDataFrame and delete DataFrame
#### Note: This is not required outside of this workbook, only changing to GeoDtaFrame to allow view on the map¶

In [6]:
# Create GeoDataFrame
grid_data["geometry"] = grid_data.apply(lambda x: "POINT (" + str(geohash.decode(x.grid)).split(", ")[1].replace(")", "") + " " + str(geohash.decode(x.grid)).split(", ")[0].replace("(", "") + ")", axis = 1)
grid_data['geometry'] = grid_data['geometry'].apply(lambda x: shapely.wkt.loads(x))
grid_geo_data = gpd.GeoDataFrame(grid_data, geometry='geometry', crs='EPSG:4326')
del grid_data

# Tilt Optimisation

### Up-tilt -> Per cell algo (another feature will take care of low coverage based up-tilting as a whole)

Note: Up-tilting is not something that should be done in large jumps, as such this algorithm identies cells to be downtilited in maximum step size of 2 degrees. The theory is to identify cells with low coverage footprints in regions of low cell interference overlap. Once identified new coverage is defined for the cell when uptilited by 1 or 2 degrees and new interfence score generated. It is up the the engineer to decide what is an acceptable trade-off between coverage and interference.

The criteria for identifying and classifying a cell as an undershooter is as follows;
1. Ingest Grid-Cell data, apply initial filter 'cell_max_distance_to_cell' <= 'max_cell_distance'
    - Dataset is now reduced to cells whos maximum coverage is <= parameter value 'max_cell_distance'
2. Create dataframe with aggregated values per cell of;
    - 'event_count'
    - 'grid_count'
    - 'interference_grid_count'
3. Calculate the percentage of interfering grids in the current cell coverage
    - 'interfecrence_grid_perc'
4. Filter aggregated dataframe to remove very low sample cells as not accurate and cells with unacceptable interference levels as uptilting will increase interference score
    - 'event_count' >= 'min_cell_event_count'
    - 'interfecrence_grid_perc' <= 'grid_percent_max_cell_grid_count'
5. Calculate coverage increase for 1 and 2 degree uptilt
6. For each cell in scope
    - Create 1000 2 degrees new serving distance ranges, this will provide the relevant grids that the cell coverage when uptilted would be expanded to
    - Find the number of bins and the number of interference bins within these ranges
    - Merge new data with existing

Tunable Parameter:
- 'min_cell_event_count' (default = 200)
    - Minmum number of samples a cell must have generated to be considered in the undershooter candidate list
- 'max_cell_distance' (default = 15000 (15km))
    - The maximum distance the cell can serve to be included in the undershooting candidate list 
- 'max_cell_grid_count' (default = 2)
    - Maximum number of cells a grid can contain before being flaged as an interferer
- 'grid_percent_max_cell_grid_count' (default = 0.2 (20%))
    - Maximum percentage of interference grids allowed for cell to be included in the undershooting candidate list

In [7]:
grid_geo_data.columns

Index(['grid_cell', 'grid', 'cell_name', 'cilac', 'avg_rsrp', 'avg_rsrq',
       'avg_sinr', 'event_count', 'grid_event_count', 'cell_event_count',
       'avg_rsrp_grid', 'avg_rsrq_grid', 'avg_sinr_grid', 'avg_rsrp_cell',
       'avg_rsrq_cell', 'avg_sinr_cell', 'perc_cell_events',
       'perc_grid_events', 'distance_to_cell', 'grid_max_distance_to_cell',
       'grid_min_distance_to_cell', 'cell_max_distance_to_cell',
       'perc_cell_max_dist', 'cell_angle_to_grid', 'grid_bearing_diff',
       'Scr_Freq', 'UARFCN', 'Bearing', 'TiltE', 'TiltM', 'SiteID', 'City',
       'Height', 'TAC', 'Band', 'Vendor', 'MaxTransPwr', 'FreqMHz', 'HBW',
       'RF_Team', 'cell_count', 'same_pci_cell_count', 'dominance',
       'dist_band', 'geometry'],
      dtype='object')

In [8]:
##################
### Parameters ###
##################
min_cell_event_count = 200
max_cell_distance = 15000
max_cell_grid_count = 2
grid_percent_max_cell_grid_count = 0.2

# Limit undershooters to have max serving distance <= max_cell_distance
grid_geo_data_dist = grid_geo_data[grid_geo_data.cell_max_distance_to_cell <= max_cell_distance][['cell_name', 'cell_max_distance_to_cell', 'event_count', 'cell_count']].copy()

# Get Aaggregated grid statistics
grid_geo_data_dist['grid_count'] = 1
grid_geo_data_dist['interference_grid_count'] = grid_geo_data_dist.apply(lambda x: 1 if x.cell_count > max_cell_grid_count else 0, axis = 1)

undershooting_initial_candidates = grid_geo_data_dist.groupby(['cell_name', 'cell_max_distance_to_cell'], as_index=False).agg({'grid_count': 'sum', 'interference_grid_count': 'sum',
                                      'event_count': 'sum'})

# Find percentage of interfering grids
undershooting_initial_candidates['interfecrence_grid_perc'] = undershooting_initial_candidates['interference_grid_count'] / undershooting_initial_candidates['grid_count']

# Filter low traffic cells and high interference grid percentages
undershooting_initial_candidates = undershooting_initial_candidates[(undershooting_initial_candidates.event_count >= min_cell_event_count) & \
                        (undershooting_initial_candidates.interfecrence_grid_perc <= grid_percent_max_cell_grid_count)].sort_values(['cell_max_distance_to_cell'], ascending = True)
# Joi with Cell GIS info
undershooting_initial_candidates = undershooting_initial_candidates.merge(gis_df[["Name", "Height", "TiltE", "TiltM"]], left_on="cell_name", right_on="Name", how="inner")

# Change column names and limit to required columns
undershooting_initial_candidates.rename(columns = {'cell_max_distance_to_cell' : 'max_ta', }, inplace = True)
undershooting_initial_candidates = undershooting_initial_candidates[['cell_name',	'max_ta', 'grid_count', 'interference_grid_count', 'interfecrence_grid_perc', 'event_count', 'Height', 'TiltE', 'TiltM']]
undershooting_initial_candidates.head()


,cell_name,max_ta,grid_count,interference_grid_count,interfecrence_grid_perc,event_count,Height,TiltE,TiltM
0,DNGJT00003A_n70_AWS-4_UL15_1,1608.766608,32,2,0.062500,1017.0,30.1752,2.0,0
1,DNGJT00005A_n70_AWS-4_UL15_3,2384.932913,45,3,0.066667,396.0,46.0248,2.0,0
2,DNDEN00282A_n70_AWS-4_UL15_2,2932.155521,73,10,0.136986,597.0,24.0792,2.0,0
3,DNGJT00005A_n70_AWS-4_UL15_1,3154.179767,18,2,0.111111,669.0,46.0248,2.0,0
4,DNDEN00417A_n71_A_1,3336.776888,113,5,0.044248,789.0,20.1168,4.0,0


In [9]:
print("Undershoting analysis results in {} potential overshooters..".format(undershooting_initial_candidates.shape[0]))

Undershoting analysis results in 123 potential overshooters..


#### Find new maximum covage range (with 1 and 2 degree uptilt)

In [10]:
def _vertical_attenuation(theta_deg: float, alpha_deg: float, hpbw_v_deg: float, sla_v_db: float) -> float:
    """3GPP parabolic attenuation in vertical plane (dB)"""
    return min(12.0 * (((theta_deg - alpha_deg) / hpbw_v_deg) ** 2), sla_v_db)

def estimate_distance_after_tilt(
    d_max_m: float,
    alpha_deg: float,
    h_m: float,
    hpbw_v_deg: float = 6.5,
    sla_v_db: float = 30.0,
    n: float = 3.5,
    delta_tilt_deg: float = 1.0
    ):
    """
    Estimate the new max coverage distance after changing electrical tilt by delta_tilt_deg.
    Returns new max distance and percentage change
    """
    if alpha_deg == 0:
        print("Cannot uptilt further, returning current max TA = {}m".format(d_max_m)) 
        return d_max_m, 0
    if d_max_m <= 0 or h_m < 0:
        raise ValueError("d_max_m must be > 0 and h_m must be >= 0")
    if hpbw_v_deg <= 0 or n <= 0:
        raise ValueError("hpbw_v_deg and n must be > 0")

    # Elevation angle from site to current edge user (deg)
    theta_e_deg = math.degrees(math.atan2(h_m, d_max_m))

    # Attenuation before/after at that direction
    A_before = _vertical_attenuation(theta_e_deg, alpha_deg, hpbw_v_deg, sla_v_db)
    A_after  = _vertical_attenuation(theta_e_deg, alpha_deg + delta_tilt_deg, hpbw_v_deg, sla_v_db)

    # Gain change at the edge direction (dB); typically negative when increasing downtilt
    deltaG_dB = -(A_after - A_before)

    # Translate to distance using log-distance model
    d_new_m = d_max_m * (10.0 ** (deltaG_dB / (10.0 * n)))

    # Geometry-only boresight intercepts
    def _bore_intercept(alpha_deg_val: float) -> float:
        if abs(alpha_deg_val) < 1e-6:
            return float('inf')
        return h_m / math.tan(math.radians(alpha_deg_val))

    bore_before = _bore_intercept(alpha_deg)
    bore_after  = _bore_intercept(alpha_deg + delta_tilt_deg)

    if d_new_m < d_max_m:
        d_new_m = d_max_m
    increase_m = d_new_m - d_max_m
    increase_pct = (increase_m / d_max_m)

    return d_new_m, increase_pct

In [11]:
undershooting_initial_candidates[['max_distance_1_degree_uptilt', 'percent_distance_increase_1_degree_downtilt']] = undershooting_initial_candidates.apply(
    lambda x: pd.Series(
        estimate_distance_after_tilt(
                                    d_max_m= x.max_ta,      # current max distance (meters)
                                    alpha_deg= x.TiltE + x.TiltM,      # current electrical downtilt (deg)
                                    h_m= x.Height,             # antenna height above UE (m)
                                    hpbw_v_deg= 6.5,     # vertical HPBW (deg) from datasheet
                                    sla_v_db= 30,        # vertical side-lobe cap (dB)
                                    n= 3.5,              # path-loss exponent
                                    delta_tilt_deg= -1.0  # tilt change (+1°)
        )
    ),
    axis=1)

undershooting_initial_candidates[['max_distance_2_degree_uptilt', 'percent_distance_increase_2_degree_downtilt']] = undershooting_initial_candidates.apply(
    lambda x: pd.Series(
        estimate_distance_after_tilt(
                                    d_max_m= x.max_ta,      # current max distance (meters)
                                    alpha_deg= x.TiltE + x.TiltM,      # current electrical downtilt (deg)
                                    h_m= x.Height,             # antenna height above UE (m)
                                    hpbw_v_deg= 6.5,     # vertical HPBW (deg) from datasheet
                                    sla_v_db= 30,        # vertical side-lobe cap (dB)
                                    n= 3.5,              # path-loss exponent
                                    delta_tilt_deg= -2.0  # tilt change (+2°)
        )
    ),
    axis=1)

undershooting_initial_candidates.head()

,cell_name,max_ta,grid_count,interference_grid_count,interfecrence_grid_perc,event_count,Height,TiltE,TiltM,max_distance_1_degree_uptilt,percent_distance_increase_1_degree_downtilt,max_distance_2_degree_uptilt,percent_distance_increase_2_degree_downtilt
0,DNGJT00003A_n70_AWS-4_UL15_1,1608.766608,32,2,0.062500,1017.0,30.1752,2.0,0,1634.549121,0.016026,1608.766608,0.000000
1,DNGJT00005A_n70_AWS-4_UL15_3,2384.932913,45,3,0.066667,396.0,46.0248,2.0,0,2420.347975,0.014850,2384.932913,0.000000
2,DNDEN00282A_n70_AWS-4_UL15_2,2932.155521,73,10,0.136986,597.0,24.0792,2.0,0,3047.162130,0.039223,3050.522266,0.040369
3,DNGJT00005A_n70_AWS-4_UL15_1,3154.179767,18,2,0.111111,669.0,46.0248,2.0,0,3233.429469,0.025125,3193.084552,0.012334
4,DNDEN00417A_n71_A_1,3336.776888,113,5,0.044248,789.0,20.1168,4.0,0,3754.272059,0.125119,4069.062774,0.219459


#### Create 1000 random sample points within the new extended TA range foreach candidate in dataframe 'undershooting_initial_candidates'

In [12]:
# ==============================
# 1) Vectorized geometry helpers
# ==============================

R_EARTH = 6_371_008.8  # meters

def _destination_from(lat0_deg, lon0_deg, dist_m, bearing_deg):
    """
    Vectorized great-circle forward solution.
    lat0_deg, lon0_deg: scalars (deg)
    dist_m, bearing_deg: arrays/scalars (meters, deg)
    Returns arrays lat2_deg, lon2_deg
    """
    lat0 = np.deg2rad(lat0_deg); lon0 = np.deg2rad(lon0_deg)
    brng = np.deg2rad(bearing_deg)
    ang  = np.asarray(dist_m, dtype=float) / R_EARTH

    sin_lat0, cos_lat0 = np.sin(lat0), np.cos(lat0)
    sin_ang,  cos_ang  = np.sin(ang),  np.cos(ang)

    lat2 = np.arcsin(sin_lat0 * cos_ang + cos_lat0 * sin_ang * np.cos(brng))
    lon2 = lon0 + np.arctan2(np.sin(brng) * sin_ang * cos_lat0, cos_ang - sin_lat0 * np.sin(lat2))
    lon2 = (lon2 + np.pi) % (2 * np.pi) - np.pi  # normalize to [-180, 180)
    return np.rad2deg(lat2), np.rad2deg(lon2)

def latlon1cell_vectorized(cell_lat_deg, cell_lon_deg, ta_m, max_ta_m, hbeam_deg, azimuth_deg, rng=None):
    """
    Vectorized rewrite of your latLon1Cell (same lobe logic, ±5% TA jitter, oval beam).
    Inputs:
      - cell_lat_deg, cell_lon_deg, hbeam_deg, azimuth_deg: scalars
      - ta_m: array of distances (meters)
      - max_ta_m: scalar (meters)
    Returns:
      (lat_deg_array, lon_deg_array)
    """
    rng = np.random.default_rng() if rng is None else rng
    ta = np.asarray(ta_m, dtype=float)
    N = ta.size

    # +/- 5% TA jitter
    ta = ta * rng.uniform(0.95, 1.05, size=N)

    back_factor = 0.10
    side_factor = 0.50
    base_beam   = 1.5 * float(hbeam_deg)

    back = ta <= (max_ta_m * back_factor)
    side = (~back) & (ta <= (max_ta_m * side_factor))

    bearings = np.empty(N, dtype=float)
    set_mask = np.zeros(N, dtype=bool)

    # Back-lobe: 30% anywhere 1..359°, else in shrinking beam
    if back.any():
        choose360 = rng.random(N) < 0.30
        m360   = back & choose360
        mbeamB = back & (~choose360)

        if m360.any():
            bearings[m360] = rng.uniform(1.0, 359.0, size=m360.sum()); set_mask[m360] = True

        if mbeamB.any():
            multiple = ta[mbeamB] / (max_ta_m * back_factor)
            beam     = base_beam - (hbeam_deg * multiple)
            u        = rng.random(mbeamB.sum())
            bearings[mbeamB] = (azimuth_deg - beam) + u * (2 * beam); set_mask[mbeamB] = True

    # Side-lobes: 50% go to side lobes (split left/right), others fall through to main
    if side.any():
        go_side = rng.random(N) <= 0.5
        m_go    = side & go_side
        if m_go.any():
            multiple = ta[m_go] / (max_ta_m * side_factor)
            beam     = base_beam - (base_beam * multiple)
            left     = rng.random(m_go.sum()) <= 0.5
            centers  = np.where(left, azimuth_deg - hbeam_deg, azimuth_deg + hbeam_deg)
            u        = rng.random(m_go.sum())
            bearings[m_go] = (centers - beam) + u * (2 * beam); set_mask[m_go] = True

    # Main lobe: everything not set yet
    m_main = ~set_mask
    if m_main.any():
        ru       = rng.uniform(0.9, 1.0, size=m_main.sum())
        ratio    = ta[m_main] / max_ta_m
        multiple = np.minimum(ru, ratio)
        beam     = base_beam - (base_beam * multiple)
        u        = rng.random(m_main.sum())
        bearings[m_main] = (azimuth_deg - beam) + u * (2 * beam)

    return _destination_from(cell_lat_deg, cell_lon_deg, ta, bearings)

# ===========================
# 2) Build undershooter grid
# ===========================

def build_undershooter_grid(undershooting_final_candidates: pd.DataFrame,
                           gis_df: pd.DataFrame,
                           n_samples: int = 1000,
                           seed: int | None = None) -> gpd.GeoDataFrame:
    """
    Parameters
    ----------
    undershooting_final_candidates : DataFrame
        Columns required: ['cell_name','max_ta','max_distance_1_degree_uptilt','max_distance_2_degree_uptilt']
    gis_df : DataFrame
        Columns required: ['Name','Latitude','Longitude','HBW','Bearing']
    n_samples : int
        Samples to generate per cell.
    seed : int | None
        RNG seed for reproducibility.

    Returns
    -------
    DataFrame
        Columns: ['cell_name','ta','lat','lon']
    """
    rng = np.random.default_rng(seed)

    # Per-cell metadata (dedupe and merge once)
    cand = (undershooting_final_candidates
            [['cell_name','max_ta','max_distance_1_degree_uptilt','max_distance_2_degree_uptilt']]
            .drop_duplicates('cell_name'))

    gis_cols = (gis_df[['Name','Latitude','Longitude','HBW','Bearing']]
                .rename(columns={'Name':'cell_name',
                                 'Latitude':'lat0','Longitude':'lon0',
                                 'HBW':'hbw','Bearing':'bearing'}))

    meta = cand.merge(gis_cols, on='cell_name', how='inner').copy()
    meta['max_dist'] = np.maximum(meta['max_distance_1_degree_uptilt'],
                                  meta['max_distance_2_degree_uptilt'])

    # ensure numeric dtypes
    num_cols = ['max_ta','max_distance_1_degree_uptilt','max_distance_2_degree_uptilt',
                'max_dist','lat0','lon0','hbw','bearing']
    meta[num_cols] = meta[num_cols].apply(pd.to_numeric, errors='coerce')
    meta = meta.dropna(subset=['max_ta','max_dist','lat0','lon0','hbw','bearing'])

    rows = []
    for row in meta.itertuples(index=False):
        low  = float(min(row.max_ta, row.max_dist))
        high = float(max(row.max_ta, row.max_dist))
        if not np.isfinite(low) or not np.isfinite(high) or high <= 0:
            continue

        # Sample TA uniformly between the bounds
        ta = rng.uniform(low, high, size=n_samples)

        # Generate lat/lon using your (vectorized) lobe model
        lats, lons = latlon1cell_vectorized(
            cell_lat_deg=float(row.lat0),
            cell_lon_deg=float(row.lon0),
            ta_m=ta,
            max_ta_m=float(row.max_dist),
            hbeam_deg=float(row.hbw),
            azimuth_deg=float(row.bearing),
            rng=rng
        )

        rows.append(pd.DataFrame({
            'cell_name': row.cell_name,
            'ta': ta,
            'lat': lats,
            'lon': lons
        }))

    if not rows:
        # Nothing generated
        return gpd.GeoDataFrame(columns=['cell_name','ta','lat','lon','geometry'], crs='EPSG:4326')

    undershooter_grid = pd.concat(rows, ignore_index=True)
    
    return undershooter_grid# gdf

# ==============================
# 3) Example usage + plotting
# ==============================

# NOTE: if your variable is actually named 'undershooting_final_candidates' in your notebook,
# rename it before calling for clarity:
# undershooting_final_candidates = undershooting_final_candidates

# Build grid (reproducible)
undershooter_grid_df = build_undershooter_grid(undershooting_initial_candidates, gis_df, n_samples=1000, seed=42)
undershooter_grid_df.head(2)

,cell_name,ta,lat,lon
0,DNGJT00003A_n70_AWS-4_UL15_1,1628.721140,39.126226,-108.524712
1,DNGJT00003A_n70_AWS-4_UL15_1,1620.081997,39.126916,-108.524656


#### Get Geohash reference and create geometry for visualisation purposes

In [13]:
# Get Geohash
undershooter_grid_df['grid'] = undershooter_grid_df.apply(lambda r: geohash.encode(r['lat'], r['lon'], precision=7), axis=1)
# Remove duplicate cell-grid pairs
undershooter_grid_df = undershooter_grid_df.drop_duplicates(subset=['cell_name', 'grid'], keep='first')
# Create geometry
undershooter_grid_df["geometry"] = undershooter_grid_df.apply(lambda x: "POINT (" + str(geohash.decode(x.grid)).split(", ")[1].replace(")", "") + " " + str(geohash.decode(x.grid)).split(", ")[0].replace("(", "") + ")", axis = 1)
undershooter_grid_df['geometry'] = undershooter_grid_df['geometry'].apply(lambda x: shapely.wkt.loads(x))
# DataFrae to GeoDataFrame
undershooter_grid_gdf = gpd.GeoDataFrame(undershooter_grid_df, geometry='geometry', crs='EPSG:4326')
undershooter_grid_gdf.head(2)

,cell_name,ta,lat,lon,grid,geometry
0,DNGJT00003A_n70_AWS-4_UL15_1,1628.721140,39.126226,-108.524712,9wfyd6w,POINT (-108.52501 39.12575)
1,DNGJT00003A_n70_AWS-4_UL15_1,1620.081997,39.126916,-108.524656,9wfyd6y,POINT (-108.52501 39.12712)


In [14]:
undershooter_grid_gdf = undershooter_grid_gdf.merge(grid_geo_data[['grid', 'cell_count']].drop_duplicates(subset=['grid'], keep='first'), on = 'grid', how = 'inner')
undershooter_grid_gdf.head()

,cell_name,ta,lat,lon,grid,geometry,cell_count
0,DNGJT00003A_n70_AWS-4_UL15_1,1628.721140,39.126226,-108.524712,9wfyd6w,POINT (-108.52501 39.12575),1
1,DNGJT00003A_n70_AWS-4_UL15_1,1620.081997,39.126916,-108.524656,9wfyd6y,POINT (-108.52501 39.12712),1
2,DNGJT00005A_n70_AWS-4_UL15_3,2391.454215,39.176414,-108.785979,9wfwueb,POINT (-108.78593 39.17656),1
3,DNDEN00282A_n70_AWS-4_UL15_2,2963.360276,40.566445,-104.976791,9xjqg0y,POINT (-104.97643 40.56633),1
4,DNDEN00282A_n70_AWS-4_UL15_2,2952.317766,40.566912,-104.980362,9xjqg0g,POINT (-104.98055 40.56633),1


#### Find interfering grids when 1/2 degree uptilt is appied

In [15]:
# Find interference grids for new coverage post uptilt
undershooter_grid_gdf['grid_count'] = 1
undershooter_grid_gdf['interference_grid_count'] = undershooter_grid_gdf.apply(lambda x: 1 if x.cell_count > max_cell_grid_count else 0, axis = 1)

# Merge with initial candidate list to add max distance with 1 degree/2 degree uptilt
undershooter_grid_gdf = undershooter_grid_gdf.merge(undershooting_initial_candidates[['cell_name', 'max_distance_1_degree_uptilt', 'max_distance_2_degree_uptilt']], on = 'cell_name', how = 'inner')

# Find grids within 1 degree of utilt
undershooter_grid_gdf_1_degree = undershooter_grid_gdf[
        (undershooter_grid_gdf.ta <= undershooter_grid_gdf.max_distance_1_degree_uptilt)].copy()

# Find grids within 2 degree of utilt
undershooter_grid_gdf_2_degree = undershooter_grid_gdf[
        (undershooter_grid_gdf.ta <= undershooter_grid_gdf.max_distance_2_degree_uptilt)].copy()

undershooting_initial_candidates_1_degree = (undershooter_grid_gdf_1_degree.groupby(['cell_name'], as_index=False)
                                            .agg({'grid_count': 'sum', 'interference_grid_count': 'sum'}))

undershooting_initial_candidates_2_degree = (undershooter_grid_gdf_2_degree.groupby(['cell_name'], as_index=False)
                                            .agg({'grid_count': 'sum', 'interference_grid_count': 'sum'}))


#### Find percentage of interfering grids

In [16]:
# Find percentage of interfering grids
undershooting_initial_candidates_1_degree['interfecrence_grid_perc_1_degree'] = undershooting_initial_candidates_1_degree['interference_grid_count'] / undershooting_initial_candidates_1_degree['grid_count']
undershooting_initial_candidates_2_degree['interfecrence_grid_perc_2_degree'] = undershooting_initial_candidates_2_degree['interference_grid_count'] / undershooting_initial_candidates_2_degree['grid_count']

undershooting_initial_candidates_1_degree = undershooting_initial_candidates_1_degree[['cell_name', 'grid_count', 'interference_grid_count', 'interfecrence_grid_perc_1_degree']]
undershooting_initial_candidates_1_degree.rename(columns = {'grid_count' : 'grid_count_1_degree', 'interference_grid_count' : 'interference_grid_count_1_degree'}, inplace = True)

undershooting_initial_candidates_2_degree = undershooting_initial_candidates_2_degree[['cell_name', 'grid_count', 'interference_grid_count', 'interfecrence_grid_perc_2_degree']]
undershooting_initial_candidates_2_degree.rename(columns = {'grid_count' : 'grid_count_2_degree', 'interference_grid_count' : 'interference_grid_count_2_degree'}, inplace = True)


#### Merge dataFrames

In [17]:
undershooting_initial_candidates = undershooting_initial_candidates.merge(undershooting_initial_candidates_1_degree, on = 'cell_name', how = 'inner')
undershooting_initial_candidates = undershooting_initial_candidates.merge(undershooting_initial_candidates_2_degree, on = 'cell_name', how = 'inner')
undershooting_initial_candidates.head()

,cell_name,max_ta,grid_count,interference_grid_count,interfecrence_grid_perc,event_count,Height,TiltE,TiltM,max_distance_1_degree_uptilt,percent_distance_increase_1_degree_downtilt,max_distance_2_degree_uptilt,percent_distance_increase_2_degree_downtilt,grid_count_1_degree,interference_grid_count_1_degree,interfecrence_grid_perc_1_degree,grid_count_2_degree,interference_grid_count_2_degree,interfecrence_grid_perc_2_degree
0,DNDEN00282A_n70_AWS-4_UL15_2,2932.155521,73,10,0.136986,597.0,24.0792,2.0,0,3047.162130,0.039223,3050.522266,0.040369,3,0,0.000000,3,0,0.000000
1,DNGJT00005A_n70_AWS-4_UL15_1,3154.179767,18,2,0.111111,669.0,46.0248,2.0,0,3233.429469,0.025125,3193.084552,0.012334,6,0,0.000000,6,0,0.000000
2,DNDEN00417A_n71_A_1,3336.776888,113,5,0.044248,789.0,20.1168,4.0,0,3754.272059,0.125119,4069.062774,0.219459,43,2,0.046512,78,9,0.115385
3,DNDEN00389A_n71_F-G_1,3687.007052,86,13,0.151163,771.0,8.8392,2.0,0,3879.622855,0.052242,3932.557905,0.066599,7,0,0.000000,8,0,0.000000
4,DNDEN00208B_n71_A_2,3799.347970,54,4,0.074074,537.0,27.1272,7.0,0,4770.503237,0.255611,5770.180447,0.518729,11,0,0.000000,11,0,0.000000


In [39]:
undershooting_final_candidates = undershooting_initial_candidates[((undershooting_initial_candidates.interfecrence_grid_perc_1_degree <= grid_percent_max_cell_grid_count) | \
                                 (undershooting_initial_candidates.interfecrence_grid_perc_2_degree <= grid_percent_max_cell_grid_count)) & \
                                 ((undershooting_initial_candidates.grid_count_1_degree >= undershooting_initial_candidates.grid_count * 0.05) | \
                                 (undershooting_initial_candidates.grid_count_2_degree >= undershooting_initial_candidates.grid_count * 0.05)) & \
                                 ((undershooting_initial_candidates.grid_count_1_degree >= 5) | \
                                 (undershooting_initial_candidates.grid_count_2_degree >= 5))].copy().reset_index()
undershooting_final_candidates.head()

,index,cell_name,max_ta,grid_count,interference_grid_count,interfecrence_grid_perc,event_count,Height,TiltE,TiltM,max_distance_1_degree_uptilt,percent_distance_increase_1_degree_downtilt,max_distance_2_degree_uptilt,percent_distance_increase_2_degree_downtilt,grid_count_1_degree,interference_grid_count_1_degree,interfecrence_grid_perc_1_degree,grid_count_2_degree,interference_grid_count_2_degree,interfecrence_grid_perc_2_degree
0,1,DNGJT00005A_n70_AWS-4_UL15_1,3154.179767,18,2,0.111111,669.0,46.0248,2.0,0,3233.429469,0.025125,3193.084552,0.012334,6,0,0.000000,6,0,0.000000
1,2,DNDEN00417A_n71_A_1,3336.776888,113,5,0.044248,789.0,20.1168,4.0,0,3754.272059,0.125119,4069.062774,0.219459,43,2,0.046512,78,9,0.115385
2,3,DNDEN00389A_n71_F-G_1,3687.007052,86,13,0.151163,771.0,8.8392,2.0,0,3879.622855,0.052242,3932.557905,0.066599,7,0,0.000000,8,0,0.000000
3,4,DNDEN00208B_n71_A_2,3799.347970,54,4,0.074074,537.0,27.1272,7.0,0,4770.503237,0.255611,5770.180447,0.518729,11,0,0.000000,11,0,0.000000
4,5,DNDEN00305A_n71_F-G_3,4153.571114,286,46,0.160839,2709.0,16.7640,5.0,0,4871.954071,0.172955,5504.967971,0.325358,86,9,0.104651,145,17,0.117241


#### Check Recommendation

In [40]:
cell_name = 'DNDEN00305A_n71_F-G_3'
m = grid_geo_data[grid_geo_data['cell_name'] == cell_name].explore(color = "blue", name="Grid")
m = undershooter_grid_gdf[(undershooter_grid_gdf.cell_name == cell_name) & (undershooter_grid_gdf.ta <= undershooting_initial_candidates[undershooting_initial_candidates.cell_name == cell_name].max_distance_1_degree_uptilt.iloc[0])].explore(m = m, color="orange", name="1 Degree Uptilt")
m = undershooter_grid_gdf[(undershooter_grid_gdf.cell_name == cell_name) & (undershooter_grid_gdf.ta > undershooting_initial_candidates[undershooting_initial_candidates.cell_name == cell_name].max_distance_1_degree_uptilt.iloc[0])].explore(m = m, color="red", name="1 Degree Uptilt")
m = gis_df[gis_df['Name'] == cell_name].explore(m = m, color="red", name="Cells")
folium.LayerControl().add_to(m)
m

## Export Results for Tableau

In [42]:
undershooting_final_candidates.to_csv("{}undershooting_candidates.csv".format(output_path))
undershooter_grid_gdf.to_csv("{}new_undershooting_bins.csv".format(output_path))